# Aprendizaje automático y aplicaciones

## Regresión (caso de estudio)

---
$A^3$ @ FI-UNER : 2021

### Sobre el conjunto de datos

En la ciudad de Ames, Iowa, se realizó el registro de diversas transacciones de compra-venta inmobiliaria desde 2006 a 2010. **El objetivo es identificar variables que permitan predecir el precio de venta de la propiedad**, con el fin de facilitar estimaciones regulatorias y ofrecer una alternativa de tasación a los ciudadanos. Es decir se desea construir un regresor que tome los atributos (o una selección de ellos) y que estime el precio de venta de la propiedad:

$$ \text{ Atributos } \rightarrow \text{Precio de venta}$$

**El dataset contiene 2919 observaciones y un gran número de variables explicativas (23 nominales, 23 ordinales, 14 numéricas discretas, y 20 numéricas contínuas)**. El dataset esta disponible en [este link](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). En ese link encontrará un archivo `data_description.txt` que explica todas las variables y sus posibles valores. También encontrará dos archivos de datos: `train.csv` y `test.csv` (cada unoarchivo con aproximadamente 1460 observaciones).
 
El objetivo es hacer el análisis exploratorio y entrenamiento de los modelos con `train.csv`, y luego usar `test.csv` para evaluar el modelo final en datos nunca vistos. El archivo `test.csv` no tiene las etiquetas (el precio de las casas) sino que se utilizará el sistema de Kaggle para enviar las predicciones y así obtener la medida de error. 
Este tipo de esquema asegura que las etiquetas de la particion de test NO influencien el desarrollo, y así evitan el sobreajuste.

### Carga de las librerías a utilizar

In [ ]:
# !pip install pandas-profiling\[notebook\] -q

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

### Lectura de los datos y exploración inicial

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train

In [ ]:
df_train.columns, df_train.shape, df_test.shape

In [ ]:
df_train.describe()

---

Las herramientas de análisis más potentes, como `pandas-profiling`, nos da información variada y algunas pistas sobre como seguir con el preprocesamiento de los datos.

In [ ]:
# ProfileReport(df_train, title="Analisis exploratorio inicial", minimal=True)  # Reporte minimo
# ProfileReport(df_train.sample(frac=.10), title="Analisis exploratorio inicial")  # Reporte con una porcion de los datos
# ProfileReport(df_train, title="Analisis exploratorio inicial", interactions={"targets": ["SalePrice"]})

ProfileReport(df_train, title="Analisis exploratorio inicial", interactions=None)

### Descripción de los datos

Si contamos con ella también es útil al momento del análisis exploratorio. Ver `data_description.txt`

Por ejemplo, podemos descubrir que:
- Unas cuantas variables usan el valor "NA" como dato válido, y pandas por defecto lo toma como `nan`.
- Hay variables categóricas que se toman como numéricas (por ejemplo MSSubClass)
- Hay variables categóricas ordinales que no se toman como tales (por ejemplo KitchenQual o GarageQual)

In [ ]:
# df_train.Alley

## Recarga de los datos con conversión de valores

In [ ]:
# Función auxiliar para convertir valores
def NA2None(cell):
    if cell == "NA":
        return "None"
    
NA_converter = {"Alley": NA2None}

df_train = pd.read_csv('train.csv', converters=NA_converter) 
df_test = pd.read_csv('test.csv', converters=NA_converter) 

In [ ]:
# df_train.Alley

In [ ]:
ProfileReport(df_train, title="Analisis exploratorio inicial", interactions=None)

### Interacción con algunas variables

In [ ]:
# GrLivArea: superficie habitable sin contar el sotano
df_train.plot.scatter(x="GrLivArea", y="SalePrice")
# OverallQual: calidad general de la vivienda
df_train.plot.scatter(x="OverallQual", y="SalePrice")
# LandSlope: inclinación del terreno 
df_train.plot.scatter(x="LandSlope", y="SalePrice");
# Neighborhood: barrio
df_train.plot.scatter(x="Neighborhood", y="SalePrice", rot=45);

### Limpieza de los datos

Si bien queremos construir un modelo basado en los datos, no siempre es bueno utilizar todos los datos tal cual están. Por ejemplo, podríamos tomar criterios como los siguientes para comenzar el modelado. Consideraciones para filtrar:
- registros de menor superficie habitable 
- operaciones anotadas como normales

In [ ]:
criteria = (df_train.GrLivArea <= 2500) & (df_train.SaleCondition == "Normal")

df_train_rev = df_train.loc[criteria]

In [ ]:
# GrLivArea: superficie habitable sin contar el sotano
df_train_rev.plot.scatter(x="GrLivArea", y="SalePrice")
# OverallQual: calidad general de la vivienda
df_train_rev.plot.scatter(x="OverallQual", y="SalePrice")
# LandSlope: inclinación del terreno 
df_train_rev.plot.scatter(x="LandSlope", y="SalePrice");
# Neighborhood: barrio
df_train_rev.plot.scatter(x="Neighborhood", y="SalePrice", rot=45);

## Distribución de `SalePrice`

In [ ]:
sns.histplot(df_train_rev.loc[:, 'SalePrice'], kde=True);

In [ ]:
# Se acerca a una distribución gaussiana?
stats.probplot(df_train_rev.loc[:, 'SalePrice'], plot=plt);

## Transformación de variables de entrada y/o salida

Si lo consideramos necesario se pueden hacer transformación de variables. Por ejemplo, se podría aplicar alguna función a la salida esperada (con el cuidado de invertir la transformación al momento de predecir los valores finales). Más adelante veremos otra estrategia para incorporarlo al pipeline.

In [ ]:
df_train_rev.loc[:, "SalePriceLog"] = np.log1p(df_train_rev.loc[:, 'SalePrice'])

In [ ]:
sns.displot(df_train_rev["SalePriceLog"], kde=True);

In [ ]:
# Se acerca a una distribución gaussiana?
res = stats.probplot(df_train_rev["SalePriceLog"], plot=plt)

---

También se podrían transformar las variables de entrada. Tener en cuenta que al hacerlo "manualmente" deberíamos aplicar lo mismo sobre el conjunto de test.

## Construcción de la matriz de entrada y el vector de salida

In [ ]:
features_col = ["GrLivArea", "OverallQual"]

X_train = df_train_rev.loc[:, features_col].values
y_train = df_train_rev.loc[:, "SalePrice"].values

results = {}

## Resultado con Regresión Lineal

[sklearn LinearRegression](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)

In [ ]:
from sklearn.linear_model import LinearRegression

regLR = LinearRegression()

In [ ]:
from sklearn.model_selection import cross_validate

scores_to_use = ("r2", "neg_mean_squared_error")

# Validación cruzada con folds y scores definidos
cv_scores = cross_validate(regLR, X_train, y_train, cv=5, scoring=scores_to_use)
results["LR"] = cv_scores

for sc in scores_to_use:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]),
          "", sep="\n")

In [ ]:
from sklearn.model_selection import cross_val_predict

# Solo realizado a los fines de graficación, no sería necesario.
cv_predicts = cross_val_predict(regLR, X_train, y_train, cv=5)
        
plt.scatter(cv_predicts, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

In [ ]:
# Si lo entrenáramos con todos los datos tendríamos una gráfica "equivalente" 
# pero sin poder analizar resultados (porque train y validation son lo mismo)
regLR.fit(X_train, y_train)
y_pred = regLR.predict(X_train)

plt.scatter(y_pred, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

## Transformación de la salida (cliping)

[sklearn TransformedTargetRegressor](https://scikit-learn.org/stable/modules/compose.html#transforming-target-in-regression)

In [ ]:
from sklearn.compose import TransformedTargetRegressor

# Regresor lineal básico con transformación de la salida
regLRclip = TransformedTargetRegressor(
    regressor=regLR, 
    func=lambda x: x,                                   # lineal
    inverse_func=lambda x: np.clip(x, 50_000, 350_000), # establece limites
    check_inverse=False
)

cv_scores = cross_validate(regLRclip, X_train, y_train, cv=5, scoring=scores_to_use)  
results["LRclip"] = cv_scores

for sc in scores_to_use:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]),
          "", sep="\n")

In [ ]:
cv_predicts = cross_val_predict(regLRclip, X_train, y_train, cv=5)
plt.scatter(cv_predicts, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

## Transformación de la salida (log exp)

[sklearn TransformedTargetRegressor](https://scikit-learn.org/stable/modules/compose.html#transforming-target-in-regression)

In [ ]:
from sklearn.compose import TransformedTargetRegressor

# Regresor lineal básico con transformación de la salida
regLRlog = TransformedTargetRegressor(
    regressor=regLR, 
    func=np.log1p,         # log(1 + x)
    inverse_func=np.expm1  # exp(x) - 1
)

In [ ]:
cv_scores = cross_validate(regLRlog, X_train, y_train, cv=5, scoring=scores_to_use)
results["LRlog"] = cv_scores

for sc in scores_to_use:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]), 
          "", sep="\n")

In [ ]:
cv_predicts = cross_val_predict(regLRlog, X_train, y_train, cv=5)
plt.scatter(cv_predicts, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

## Transformación de las variables de entrada

In [ ]:
features_col = ["GrLivArea", "OverallQual", "LandSlope", "Neighborhood"]

X_train = df_train_rev.loc[:, features_col].values
X_test = df_test.loc[:, features_col].values
y_train = df_train_rev.loc[:, "SalePrice"].values

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler

# Se definen las transformaciones para las entradas
preprocessor = make_column_transformer(
#     ("passthrough", [0, 1, 2, 3]),
    (StandardScaler(), [0, 1]),
#     (MinMaxScaler(), [0, 1]),
    (OrdinalEncoder(categories=[("Gtl", "Mod", "Sev")]), [2]),
    (OneHotEncoder(), [3]),
    remainder='passthrough'
)

# Display resultados solo a los fines de ejemplificar
display("original", X_train[212:215])
display("transformed", preprocessor.fit_transform(X_train[212:215]))

## Apilado del preprocesamiento y el regresor con transformación de salida

In [ ]:
from sklearn.pipeline import make_pipeline

preLRlog = make_pipeline(
    preprocessor,
    regLRlog
)

In [ ]:
from sklearn import set_config

set_config(display='diagram')

preLRlog

In [ ]:
cv_scores = cross_validate(preLRlog, X_train, y_train, cv=5, scoring=scores_to_use)
results["preLRlog"] = cv_scores

for sc in scores_to_use:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]), 
          "", sep="\n")

In [ ]:
cv_predicts = cross_val_predict(preLRlog, X_train, y_train, cv=5)
plt.scatter(cv_predicts, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

## Optar por otro tipo de regresor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

preRFlog = make_pipeline(
    preprocessor,
    RandomForestRegressor(n_estimators=50, max_depth=4, random_state=42)
)

In [ ]:
cv_scores = cross_validate(preRFlog, X_train, y_train, cv=5, scoring=scores_to_use[1:])
results["preRFlog"] = cv_scores

# Ya no podemos usar R^2 como métrica, el modelo no es lineal
for sc in scores_to_use[1:]:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]), 
          "", sep="\n")

In [ ]:
cv_predicts = cross_val_predict(preRFlog, X_train, y_train, cv=5)
plt.scatter(cv_predicts, y_train)
plt.xlabel("Valores Estimados")
plt.ylabel("Valores Verdaderos");

## Optimizar un hiperparámetro del modelo

In [ ]:
preRFlog.steps

In [ ]:
from sklearn.model_selection import GridSearchCV

param_to_explore = {"n_estimators": [5, 50, 500]}

# En el pipeline mismo se define una busquerda de los hiperparámetros
preRFcvlog = make_pipeline(
    preprocessor,
    GridSearchCV(RandomForestRegressor(max_depth=4, random_state=42), 
                 param_grid=param_to_explore, verbose=2, cv=2)
)

In [ ]:
cv_scores = cross_validate(preRFcvlog, X_train, y_train, cv=5, scoring=scores_to_use[1:])
results["preRFcvlog"] = cv_scores

# Ya no podemos usar R^2 como métrica, el modelo no es lineal
for sc in scores_to_use[1:]:
    print(sc, 
          cv_scores[f"test_{sc}"],
          np.mean(cv_scores[f"test_{sc}"]), 
          "", sep="\n")

In [ ]:
metric_to_plot = "test_neg_mean_squared_error"
results_to_plot = {}

for reg_name in results:
    results_to_plot[reg_name] = results[reg_name][metric_to_plot] * -1.0

results_to_plot = pd.DataFrame(results_to_plot)
results_to_plot

In [ ]:
results_to_plot.melt()

In [ ]:
sns.barplot(x="variable", y="value" ,data=results_to_plot.melt());

## Predicciones sobre el conjunto de test

Considerando los resultados obtenidos, seleccionamos el mejor modelo, lo entrenamos con la partición de train y predecimos sobre la partición de test.

In [ ]:
preLRlog.fit(X_train, y_train)  # Entrenamiento con todos los datos de train

y_pred = preLRlog.predict(X_test)  # Predicciones sobre todos los de test
y_pred

In [ ]:
# Generar archivo para subir a kaggle

submission_df = pd.DataFrame({"Id": df_test.loc[:, "Id"], 
                              "SalePrice": y_pred})

submission_df.to_csv("submission_preLRlog.csv", index=False)

# Resultado obtenido: 
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques
# 0.17366 Root mean squared logarithmic error

submission_df